In [1]:
#Same as the first copy, this time I'm going to use Hailans data, and see if it makes a difference.

%pip install tensorflow
%pip install --upgrade Pillow
%pip install -U scikit-image

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
import os
import glob
import numpy as np 
import random
import cv2
import tensorflow as tf
from PIL import Image
from skimage import io
import sagemaker
import boto3
import pandas as pd
import numpy
from torch.utils.data import Dataset, DataLoader

In [3]:
class Net(nn.Module):


  def __init__(self):

    super(Net, self).__init__()

    input_width = 32
    input_height = 32


    self.net = nn.Sequential()


    #Activation map of size: 3 * input_width * input_height
    self.net.add_module('cv1', nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding =  1, dilation = 1))

    #Activation map of size: 64 * input_widht * input_height
    self.net.add_module('r11', nn.ReLU())

    #Activation map of size: 64 * input_width * input_height
    self.net.add_module('cv2', nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #Activation map of size 128* input_width * input_height
    self.net.add_module('rl2', nn.ReLU())

    #Activation map of size: 128 * input_width/2 * input_height/2
    self.net.add_module('mp1', nn.MaxPool2d(kernel_size = 2, stride = None, padding = 0, dilation = 1))

    #Activation map of size: 128 * input_width/2 * input_height/2
    self.net.add_module('cv3', nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #Activation map of size: 256 * input_width/2 * input_height/2
    self.net.add_module('rl3', nn.ReLU())

    #Activation map of size 256 * input_width/2 * input_height /2
    self.net.add_module('cv4', nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #Activation map of size 512 * input_width/2 * input_height/2
    self.net.add_module('rl4', nn.ReLU())

    #Activation map of size 512 * input_width/2 * input_height/2
    self.net.add_module('mp2', nn.MaxPool2d(kernel_size = 2, stride = None, padding = 0, dilation = 1))

    #Activation map of size 512 * input_width/4 * input_height/4
    self.net.add_module('cv5', nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #Activation map of size 512 * input_width/4 * input_height/4
    self.net.add_module('rl5', nn.ReLU())

    #activation map of size 512 * input_width/4 * input_height/4
    #self.net.add_module('cv6', nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #activation map of size 512 * input_width/4 * input_height/4
    #self.net.add_module('rl6', nn.ReLU())



    #Multilayer perceptron
    self.net.add_module('dp1', nn.Dropout2d(p = 0.25))
    self.net.add_module('fl1', nn.Flatten())
    self.net.add_module('fc1', nn.Linear(in_features = 2768896, out_features = 128))
    self.net.add_module('rl7', nn.ReLU())
    self.net.add_module('fc2', nn.Linear(in_features = 128, out_features = 4))
    self.net.add_module('sm1', nn.LogSoftmax(dim = 1))

  def forward(self, x):
    return self.net(x.float())

model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.4)

# Let's define a Loss function

lossfun = nn.CrossEntropyLoss()  # Use nn.CrossEntropyLoss with softmax

In [4]:

%matplotlib inline

from torchvision import datasets, transforms
import os

imgs = datasets.ImageFolder('Data/Padded', transform = transforms.ToTensor())

f = open("Data/Target/y_target.txt")

path = "data2/train"

train_path = os.path.join(path, 'images', '*')
train_paths = sorted(glob.glob(train_path)) 

path = "data2/test"
test_path = os.path.join(path, 'images', '*')
test_paths = sorted(glob.glob(test_path))

path = "data2/val"
val_path = os.path.join(path, 'images', '*')
val_paths = sorted(glob.glob(val_path))

len(train_paths)


1801

In [5]:
for i in range(5):
    img = io.imread(train_paths[i])
    print(img.shape)
    

(416, 416, 3)
(416, 416, 3)
(416, 416, 3)
(416, 416, 3)
(416, 416, 3)


In [6]:
#Attempting to make a custom dataset, so that we can use the below training algorthm. 

class TrashDataset():
    def __init__(self, target_vector_file, train, transform = None):
        dataset = {}
        
        self.train = train
        
        if train:
            target_file = open("data2/Target/y_train_with_name.txt", "r")
        else:
            target_file = open("data2/Target/y_test_with_name.txt", "r")
        img_name = target_file.readline()
        i = 0
        classifications = []
        img_names = []
        while(img_name != ""):
            img_names.append(str(img_name).strip())
            classifications.append(str(target_file.readline()).strip())
            img_name = target_file.readline()
        target_file.close()
        
        dataset = []
        
        for i in range(len(img_names)):
            dataset.append( (img_names[i], classifications[i]))
        
        self.dataframe = pd.DataFrame(dataset)
        #self.dataframe = self.dataframe.sample(frac = 1, random_state = 42)
        
        print(self.dataframe)
        self.transform = transform
        
        
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__ (self, index):
        if torch.is_tensor(index):
            idx = idx.tolist(index)
        img_name = self.dataframe.iloc[index, 0]
        if self.train:
            img = io.imread("data2/train/images/" + img_name[:-5] + ".jpg")
            #print(img_name[:-5], ".jpg")
        else:
            img = io.imread("data2/test/images/" + img_name[:-5] + ".jpg")
        classification = self.dataframe.iloc[index, 1]
        sample = {"image": img, "classification": classification}
        sample = self.transform(sample)
        return sample
        

        
class ToTensor(object):
    #Class to turn nparrays to tensor
    def __call__(self, sample):
        image, classification= sample["image"], sample["classification"]
        image = image.transpose((2, 0, 1))
        return {"image": torch.from_numpy(image), "classification": int(classification)}

In [7]:
train_dataset = TrashDataset("Data/Target/y_target_with_name.txt", train = True, transform = ToTensor())
test_dataset = TrashDataset("Data/Target/y_target_with_name.txt", train = False, transform = ToTensor())


                                                      0  1
0     003a1a43-509b-4ce7-922b-2a6851e4ef41_1296e748-...  3
1     003a1a43-509b-4ce7-922b-2a6851e4ef41_1296e748-...  3
2     003a1a43-509b-4ce7-922b-2a6851e4ef41_1296e748-...  3
3     003d0583-7137-402d-82a4-de182b2287af_7e6dee91-...  1
4     003d0583-7137-402d-82a4-de182b2287af_7e6dee91-...  1
...                                                 ... ..
1796  ffec0471-dd09-4e0c-b34a-b66f8fdb700d_4c82e9b6-...  0
1797  fff9a9bc-350a-454c-a9e5-254d388786b6_b7520d69-...  2
1798  fff9a9bc-350a-454c-a9e5-254d388786b6_b7520d69-...  2
1799  fff9a9bc-350a-454c-a9e5-254d388786b6_e2481186-...  2
1800  fff9a9bc-350a-454c-a9e5-254d388786b6_e2481186-...  2

[1801 rows x 2 columns]
                                                    0  1
0   03d2b09f-4b13-4259-af6d-74ca47e1b6f5_1d0c9370-...  2
1   0be28175-fb34-4aa9-9b28-c34754eed48a_dcafe8ad-...  0
2   0c826e4d-3b5f-41da-8d0c-acbf92971e63_a7b674ba-...  1
3   1bd3fd7c-e94a-48b7-bedc-89b918d26f4

In [ ]:
#Training the model, (here we go!)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 0)

def train(model, train_loader, epochs):
    model.train()

    # Define train epochs
    
    test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

    for epoch in range(epochs):
        

        # iterate through train dataset
        epoch_loss = []
        epoch_accu = []

        for batch_idx, sample in enumerate(train_loader):
            
            #data, target = data.to(device), target.to(device)
            
            data = sample["image"]
            target = sample["classification"]

            #print(data.shape)

            # get output
            output = model(data)

            # compute loss function
            loss = lossfun(output, target)
            
            # backward pass
            optimizer.zero_grad()
            loss.backward()

            # run optimizer 
            optimizer.step()

            # bookkeeping
            accuracy = (output.argmax(-1) == target).float().mean()
            epoch_loss.append(loss.item())
            epoch_accu.append(accuracy.item())

            #if batch_idx % 12 == 0 and batch_idx == 0:
                #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2f}'.format(
                    #epoch+1, batch_idx*32, len(train_loader.dataset),
                    #100. * batch_idx / len(train_loader), sum(epoch_loss)/len(epoch_loss), sum(epoch_accu)/len(epoch_accu))
        
        print('Train Epoch: {}\tAverage Loss: {:.6f}\tAverage Accuracy: {:.2f}'.format(
            epoch+1, sum(epoch_loss)/len(epoch_loss), sum(epoch_accu)/len(epoch_accu)))
        
        test(test_loader)
            
    # save network
    
    return epoch_loss, epoch_accu

epoch_loss, epoch_accu = train(model, train_loader, epochs=26)


Train Epoch: 1	Average Loss: 0.389072	Average Accuracy: 0.86
starting to test...
Average Loss: 2.010589	Average Accuracy: 0.25
Train Epoch: 2	Average Loss: 0.240675	Average Accuracy: 0.93
starting to test...
Average Loss: 2.026467	Average Accuracy: 0.58
Train Epoch: 3	Average Loss: 0.052259	Average Accuracy: 0.99
starting to test...
Average Loss: 2.679913	Average Accuracy: 0.55
Train Epoch: 4	Average Loss: 0.124101	Average Accuracy: 0.97
starting to test...
Average Loss: 2.075985	Average Accuracy: 0.58
Train Epoch: 5	Average Loss: 0.018768	Average Accuracy: 1.00
starting to test...
Average Loss: 2.263943	Average Accuracy: 0.57
Train Epoch: 6	Average Loss: 0.008693	Average Accuracy: 1.00
starting to test...
Average Loss: 2.663777	Average Accuracy: 0.57
Train Epoch: 7	Average Loss: 0.005492	Average Accuracy: 1.00
starting to test...
Average Loss: 2.762723	Average Accuracy: 0.58
Train Epoch: 8	Average Loss: 0.003638	Average Accuracy: 1.00
starting to test...
Average Loss: 2.892420	Average

In [ ]:
def test(test_loader):
    
    total_loss = []
    total_accu = []
    
    
    
    print("starting to test...")
    
    for batch_idx, sample in enumerate(test_loader):
            

            #data, target = data.to(device), target.to(device)
            
            data = sample["image"]
            target = sample["classification"]

            #print(data.shape)

            # get output
            output = model(data)

            # compute loss function
            loss = lossfun(output, target)

            # bookkeeping
            accuracy = (output.argmax(-1) == target).float().mean()
            total_loss.append(loss.item())
            total_accu.append(accuracy.item())
            

    print('Average Loss: {:.6f}\tAverage Accuracy: {:.2f}'.format(
            sum(total_loss)/len(total_loss), sum(total_accu)/len(total_accu)))
            
    
    return total_loss, total_accu

test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

#test(test_loader)

In [ ]:
#Train attempt 2

#Set up permissions and enviroment variables. 

import sagemaker
from sagemaker import get_execution_role
role = get_execution_role
print(role)

sess = sagemaker.Session()
bucket = "e-team6"
prefix = "ic-fulltraining"

In [12]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, "image-classification", repo_version = "latest")



The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


In [13]:
#Data Preperation

import boto3 

#Function to transfer data to s3 for training. 

def upload_to_s3(channel, file):
    s3 = boto3.resource("s3")
    data = open(file, "rb")
    key = channel + "/" + file
    s3.Bucket(bucket).put_object(Key = key, Body = data)

In [14]:
#Split data into to channels -- train and test

s3train = "s3://{}/{}/train".format(bucket, prefix)
s3validation = "s3://{}/{}/validation".format(bucket, prefix)

In [15]:
#Upload images into validation and training channels

In [16]:
s3_output_location = "s3://{}/{}/output".format(bucket, prefix)

ic = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count = 1, 
    train_instance_type = "m1.p2.xlarge",
    train_volume_size = 50, 
    train_max_run = 360000,
    input_model = "file",
    output_path = s3_output_location, 
    sagemaker_session = sess

    )

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [17]:
#Set up hyperparmeters in the algorithm.

ic.set_hyperparameters(
    num_layers = 18, 
    image_shape = "3, 5312, 4032",
    num_classes = 4, 
    num_training_examples = 400,
    mini_batch_size = 32,
    epochs = 10,
    learning_rate = 0.1,
    top_k = 2#I have no idea what this parameter does
    )

In [18]:
train_data = sagemaker.inputs.TrainingInput(
    s3train, 
    distribution = "FullyReplicated",
    content_type = "image/jpeg",
    s3_data_type = "S3Prefix",
    )

validation_data = sagemaker.inputs.TrainingInput(
    s3validation, 
    distribution = "FullyRepliatied",
    content_type = "image/jpeg", 
    s3_data_type = "S3Prefix"
    )

data_channels = {"train": train_data, "validation": validation_data}

In [19]:
import os
import glob

PATH = "train"
image_path = os.path.join(PATH, 'images', '*')
image_paths = sorted(glob.glob(image_path)) 

for path in image_paths:
    upload_to_s3("train", path)
    
PATH = "valid"    
image_path = os.path.join(PATH, "images", "*")
imagePaths = sorted(glob.glob(image_path))

for path in image_paths:
    upload_to_s3("validation", path)
    

In [20]:
ic.fit(inputs=data_channels, logs=True)

TypeError: argument of type 'function' is not iterable